In [3]:
%pwd

'c:\\Users\\sujee\\Desktop\\Project\\ML-Project-With-ML-Flow\\Notebooks'

In [4]:
import os

os.chdir("../")

In [5]:
%pwd

'c:\\Users\\sujee\\Desktop\\Project\\ML-Project-With-ML-Flow'

In [28]:
import os

os.environ["MLFLOW_TRACKING_URI"] = "https://dagshub.com/Sujeet2003/ML-Project-With-ML-Flow.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"] = "Sujeet2003"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "64c0a69eb84c6b0b4a56e46d0044dd34370fd8eb"


In [29]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str

In [30]:
from MLProject.constants import *
from MLProject.utils.common import read_yaml, create_directories

In [31]:
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH, schema_filepath=SCHEMA_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir = config.root_dir,
            test_data_path = config.test_data_path,
            model_path = config.model_path,
            all_params = params,
            metric_file_name = config.metric_file_name,
            target_column = schema.name,
            mlflow_uri = "https://dagshub.com/Sujeet2003/ML-Project-With-ML-Flow.mlflow",
        )
        return model_evaluation_config

In [32]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib
from MLProject.utils.common import save_json

In [33]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def eval_metrics(self, y_test, y_pred):
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        r2 = r2_score(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)

        return rmse, mae, r2
    
    def login_to_mlflow(self):
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        X_test = test_data.drop(columns=['quality', 'Id'])
        y_test = test_data[[self.config.target_column]]

        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_uri_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            y_pred = model.predict(X_test)
            rmse, mae, r2 = self.eval_metrics(y_test, y_pred)

            scores = {
                'rmse': rmse,
                'mae': mae,
                'r2_score': r2,
            }
            save_json(path=Path(self.config.metric_file_name), data=scores)

            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("RMSE", rmse)
            mlflow.log_metric("MAE", mae)
            mlflow.log_metric("R2 Score", r2)

            if tracking_uri_type_store != "file":
                mlflow.sklearn.log_model(model, "model", registered_model_name="ElasticNetModel")
            else:
                mlflow.sklearn.log_model(model, "model")

In [34]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.login_to_mlflow()
except Exception as e:
    raise e

[2024-12-22 23:17:34,445: INFO: common: yaml file config\config.yaml loaded successfully]
[2024-12-22 23:17:34,448: INFO: common: yaml file params.yaml loaded successfully]
[2024-12-22 23:17:34,451: INFO: common: yaml file schema.yaml loaded successfully]
[2024-12-22 23:17:34,452: INFO: common: created directory at artifacts]
[2024-12-22 23:17:34,454: INFO: common: created directory at artifacts/model_evaluation]
[2024-12-22 23:17:35,063: INFO: common: json file saved at: artifacts\model_evaluation\metrics.json]


Registered model 'ElasticNetModel' already exists. Creating a new version of this model...
2024/12/22 23:17:45 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: ElasticNetModel, version 3
Created version '3' of model 'ElasticNetModel'.
